# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [ ]:
# Import numpy and pandas
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Challenge 1 

In this lab, we will look at a dataset of sensor data from a cellular phone. The phone was carried in the subject's pocket for a few minutes while they walked around.

To load the data, run the code below.

In [ ]:
# Run this code:

sensor = pd.read_csv('../sub_1.csv')
sensor.drop(columns=['Unnamed: 0'], inplace=True)

Examine the data using the `head` function.

In [ ]:
# Your code here:

sensor.head()

Check whether there is any missing data. If there is any missing data, remove the rows containing missing data.

In [ ]:
# Your code here:

sensor.isnull().sum().sort_values(ascending=False)

How many rows and columns are in our data?

In [ ]:
# Your code here:

len(sensor.columns)

To perform time series analysis on the data, we must change the index from a range index to a time series index. In the cell below, create a time series index using the `pd.date_range` function. Create a time series index starting at 1/1/2018 00:00:00 and ending at 1/1/2018 00:29:10. The number of periods is equal to the number of rows in `sensor`. The frequency should be set to `infer`.

In [ ]:
# Your code here:

infer = pd.date_range(start='1/1/2018 00:00:00', end='1/1/2018 00:29:10', periods=len(sensor))

Assign the time series index to the dataframe's index.

In [ ]:
# Your code here:

sensor = sensor.set_index(infer)

Our next step is to decompose the time series and evaluate the patterns in the data. Load the `statsmodels.api` submodule and plot the decomposed plot of `userAcceleration.x`. Set `freq=60` in the `seasonal_decompose` function. Your graph should look like the one below.

![time series decomposition](../images/tsa_decompose.png)

In [ ]:
# Your code here:

decompose = sm.tsa.seasonal_decompose(sensor['userAcceleration.x'], freq=60)
decompose.plot().show()

Plot the decomposed time series of `rotationRate.x` also with a frequency of 60.

In [ ]:
decompose = sm.tsa.seasonal_decompose(sensor['rotationRate.x'], freq=60)
decompose.plot().show()

The next step will be to test both variables for stationarity. Perform the Augmented Dickey Fuller test on both variables below.

In [ ]:
# Your code here:
from statsmodels.tsa.stattools import adfuller

test_userAcceleration = adfuller(sensor['userAcceleration.x'])
test_rotationRate = adfuller(sensor['rotationRate.x'])

What are your conclusions from this test?

In [ ]:
# Your conclusions here:

